In [2]:
from geneformer import TranscriptomeTokenizer

In [10]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type", "organ_major": "organ"}, nproc=16)
tk.tokenize_data(
    "train_dataset",
    "tokenized_train_dataset",
    "tokenized",
    file_format = "h5ad"  
)

tk.tokenize_data(
    "test_dataset",
    "tokenized_test_dataset",
    "tokenized",
    file_format = "h5ad"  
)

Tokenizing train_dataset/CD4_trainset_prepped.h5ad


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  6.53it/s]
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/merge.py:1434: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/anndata.py:787: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  getattr(self, attr).index = value
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

train_dataset/CD4_trainset_prepped.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.
Tokenizing test_dataset/CD4_testset_prepped.h5ad


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.39it/s]
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/merge.py:1434: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/anndata.py:787: ImplicitModificationWarning: Trying to modify index of attribute `.obs` of view, initializing view as actual.
  getattr(self, attr).index = value
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/anndata/_core/anndata.py:1758: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/geneformer/tokenizer.py:495: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future v

test_dataset/CD4_testset_prepped.h5ad has no column attribute 'filter_pass'; tokenizing all cells.
Creating dataset.


In [1]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats
from geneformer import EmbExtractor

/c4/home/tagashe/jdmanalysis/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cell_states_to_model = {
    "state_key": "disease_group",
    "start_state": "Active",      # simulate perturbation in active disease
    "goal_state": "HC",           # see if it shifts toward healthy control
    "alt_states": ["Inactive", "TNJDM"]  # optional: other labels not of primary interest
}


In [30]:
embex = EmbExtractor(model_type="Pretrained",
        num_classes=2,
        emb_mode="cell",
        max_ncells=1000,
        emb_layer=-1,
        emb_label=None,
        labels_to_plot=["disease_group", "cell_type"],
                    summary_stat = "exact_mean")

In [ ]:
state_embs_dict = embex.get_state_embs(cell_states_to_model,
                                       "../Geneformer/gf-20L-95M-i4096",
                                       input_data_file="tokenized_train_dataset/tokenized.dataset",
                                       output_directory="train_embeddings",
                                       output_prefix="embs")

In [3]:
isp = InSilicoPerturber(
    perturb_type="delete",
    genes_to_perturb="all",
    combos=0,
    model_type="CellClassifier",
    num_classes=2,                    # or 3 if you're using Active, Inactive, HC
    emb_mode="cls",                   # use CLS token for classification
    cell_emb_style="mean_pool",      # ignored for CLS mode
    max_ncells=500,
    emb_layer=0,
    forward_batch_size=64,
    nproc=8
)

In [40]:
import torch
torch.cuda.empty_cache()


In [6]:
isp.perturb_data("../Geneformer/gf-12L-95M-i4096", # example 30M fine-tuned model
                 "tokenized_train_dataset/tokenized.dataset",
                 "pertubations",
                 "perturb")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../Geneformer/gf-12L-95M-i4096 and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.14it/s]
                                                                                                                                                                     
  0%|                                                                                                                                         | 0/45 [00:00<?, ?it/s]

  0%|                                                                                                                                        

OutOfMemoryError: CUDA out of memory. Tried to allocate 718.00 MiB. GPU 0 has a total capacity of 15.72 GiB of which 627.56 MiB is free. Process 2281378 has 358.00 MiB memory in use. Process 2283570 has 358.00 MiB memory in use. Process 3200602 has 204.00 MiB memory in use. Process 4147481 has 7.28 GiB memory in use. Process 243005 has 422.00 MiB memory in use. Including non-PyTorch memory, this process has 6.50 GiB memory in use. Of the allocated memory 5.24 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# extracts data from intermediate files and processes stats to output in final .csv
ispstats.get_stats("pertubations", # this should be the directory 
                   None,
                   "pertubation_stats",
                   "perturb_stats")